## Question 1: Image Preprocessing for Inference (PyTorch)
Problem: Write a function to load an image and preprocess it for inference.


In [ ]:
from PIL import Image
import torch
from torchvision import transforms

def preprocess_image(image_path):
    transform_pipeline = transforms.Compose([
        # Resize image
        transforms.Resize((224, 224)),  
        # Convert to tensor
        transforms.ToTensor(),           
         # Normalize with ImageNet stats
        transforms.Normalize(               
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    # Loaded image
    img = Image.open(image_path).convert("RGB")  
    # Applyed transforms
    img = transform_pipeline(img) 
     # Added batch dimension
    img = img.unsqueeze(0)                      
    
    return img

## Question 2: Predict on New Image with a Trained Model
Problem: Perform prediction and get the class label.


In [ ]:
import torch

def predict_image(model, image_tensor, class_names):
    # Set model to eval mode
    model.eval()          
    # Disable gradient calculation
    with torch.no_grad():                            
        outputs = model(image_tensor)                
        _, predicted = torch.max(outputs, 1)          
        class_label = class_names[predicted.item()]   
    return class_label


In [ ]:
image = preprocess_image("test.jpg")
label = predict_image(model, image, class_names)
print("Predicted Class:", label)

## Question 3: Build a CNN to classify CIFAR-10 images (PyTorch)
Problem: Create a CNN model that classifies images from the CIFAR-10 dataset with accuracy above 60%.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CIFAR10_CNN(nn.Module):
    def __init__(self):
        super(CIFAR10_CNN, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## Question 4: Identify Overfitting from Training Logs and Solve It
Problem: You notice the training accuracy increases but validation accuracy stagnates. Modify the model using dropout and early stopping. (use mnist dataset)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Load MNIST dataset
(x_train, y_train), (x_val, y_val) = tf.keras.datasets.mnist.load_data()

# Preprocessing
x_train = x_train.reshape(-1, 28, 28, 1).astype("float32") / 255.0
x_val   = x_val.reshape(-1, 28, 28, 1).astype("float32") / 255.0

# Build CNN model with Dropout
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    layers.MaxPooling2D((2,2)),
     # Dropout to reduce overfitting
    layers.Dropout(0.25),                     

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),                     

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),                       
    layers.Dense(10, activation='softmax')
])

# Early Stopping callback
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train with EarlyStopping
history = model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(x_val, y_val),
    callbacks=[early_stop]
)


## Question 5: Transfer Learning with Pretrained VGG16 (Cats vs Dogs)
Problem: Use VGG16 for binary classification with fine-tuning


In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

# 1. Loaded VGG16 without top
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
# Freeze all layers initially
base_model.trainable = False      

# 2. Added custom classifier
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

# 3. Compiled
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 4. Data loaders (simple)
train_gen = ImageDataGenerator(rescale=1./255)
val_gen = ImageDataGenerator(rescale=1./255)

train_data = train_gen.flow_from_directory("train/", target_size=(224, 224), batch_size=32, class_mode='binary')
val_data   = val_gen.flow_from_directory("val/", target_size=(224, 224), batch_size=32, class_mode='binary')

# 5. Trained (initial training)
model.fit(train_data, epochs=5, validation_data=val_data)

# 6. Fine-tuned last layers of VGG16
base_model.trainable = True
for layer in base_model.layers[:-4]:
    layer.trainable = False

# 7. Recompiled and trained again
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(train_data, epochs=3, validation_data=val_data)
